In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from metal.mmtl.trainer import MultitaskTrainer
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.slicing.slice_model import SliceModel

In [3]:
SEED = 1

### Initialize normal payloads

In [4]:
task_kwargs = {
    "dl_kwargs": {"batch_size": 8},
    "freeze_bert":False,
    "bert_model": 'bert-base-cased',
    "max_len": 200,
    "attention": False
}
task_names = ["RTE"]

In [5]:
%%time

# Create tasks and payloads
tasks, payloads = create_glue_tasks_payloads(task_names, **task_kwargs)

Using random seed: 292747
Loading RTE Dataset



CPU times: user 12.6 s, sys: 900 ms, total: 13.5 s
Wall time: 14.5 s


In [6]:
tasks, payloads

([ClassificationTask(name=RTE, loss_multiplier=1.00)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=train),
  Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=valid),
  Payload(RTE_test: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=test)])

### Initialize slice payloads

In [7]:
# Create tasks and payloads
task_kwargs.update({"slice_dict": {
    "RTE": ["dash_semicolon", "more_people", "BASE"]}
})
task_kwargs['attention'] = None

tasks_slice, payloads_slice = create_glue_tasks_payloads(
    task_names, **task_kwargs
)

Using random seed: 757595
Loading RTE Dataset



Added label_set with 2490/2490 labels for task RTE_slice:dash_semicolon:ind to payload RTE_train.
Added label_set with 1003/2490 labels for task RTE_slice:dash_semicolon:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:more_people:ind to payload RTE_train.
Added label_set with 64/2490 labels for task RTE_slice:more_people:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:BASE:ind to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:BASE:pred to payload RTE_train.
Added label_set with 277/277 labels for task RTE_slice:dash_semicolon:ind to payload RTE_valid.
Added label_set with 116/277 labels for task RTE_slice:dash_semicolon:pred to payload RTE_valid.
Added label_set with 277/277 labels for task RTE_slice:more_people:ind to payload RTE_valid.
Added label_set with 12/277 labels for task RTE_slice:more_people:pred to payload RTE_valid.
Added label_set with 277/277 labels for task RTE_slice:BA

In [8]:
tasks_slice, payloads_slice

([ClassificationTask(name=RTE, loss_multiplier=1.00),
  ClassificationTask(name=RTE_slice:dash_semicolon:ind, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:dash_semicolon:pred, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:more_people:ind, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:more_people:pred, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:BASE:ind, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:BASE:pred, loss_multiplier=0.17)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolon:ind': 'RTE_slice:dash_semicolon:ind', 'RTE_slice:dash_semicolon:pred': 'RTE_slice:dash_semicolon:pred', 'RTE_slice:more_people:ind': 'RTE_slice:more_people:ind', 'RTE_slice:more_people:pred': 'RTE_slice:more_people:pred', 'RTE_slice:BASE:ind': 'RTE_slice:BASE:ind', 'RTE_slice:BASE:pred': 'RTE_slice:BASE:pred'}], split=train),
  Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolo

In [9]:
payloads_slice[0].data_loader.dataset.labels['RTE_slice:dash_semicolon:pred'].view(1, -1)

tensor([[0, 0, 0,  ..., 0, 1, 0]])

In [10]:
payloads_slice[0].data_loader.dataset.labels['RTE_slice:dash_semicolon:ind'].view(1, -1)

tensor([[2, 2, 2,  ..., 2, 1, 2]], dtype=torch.uint8)

### Initialize and train baseline model 

In [11]:
model = MetalModel(tasks, seed=SEED, verbose=False)

In [12]:
%%time
trainer = MultitaskTrainer(seed=SEED)
trainer.train_model(
    model,
    payloads,
    checkpoint_metric="RTE/RTE_valid/RTE_gold/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=5e-5,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=10,
    progress_bar=True,
    checkpoint_best=True,
    checkpoint_cleanup=False,
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 312 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/zzweng/metal/logs/2019_04_22/15_30_04/config.json


[0.10 epo]: RTE:[RTE_train/RTE_gold/loss=7.03e-01, RTE_valid/RTE_gold/accuracy=5.34e-01] model:[train/all/loss=7.03e-01, train/all/lr=5.00e-05]
Saving model at iteration 0.10 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.534
[0.21 epo]: RTE:[RTE_train/RTE_gold/loss=7.14e-01, RTE_valid/RTE_gold/accuracy=4.73e-01] model:[train/all/loss=7.14e-01, train/all/lr=5.00e-05]
[0.31 epo]: RTE:[RTE_train/RTE_gold/loss=6.80e-01, RTE_valid/RTE_gold/accuracy=5.38e-01] model:[train/all/loss=6.80e-01, train/all/lr=5.00e-05]
Saving model at iteration 0.31 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.538
[0.41 epo]: RTE:[RTE_train/RTE_gold/loss=7.07e-01, RTE_valid/RTE_gold/accuracy=5.70e-01] model:[train/all/loss=7.07e-01, train/all/lr=5.00e-05]
Saving model at iteration 0.41 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.570
[0.51 epo]: RTE:[RTE_train/RTE_gold/loss=6.84e-01, RTE_valid/RTE_gold/accuracy=5.56e-01] model:[train/all/loss=6.84e-01, train/all/lr=5.00e-05]
[0.62 e

[1.03 epo]: RTE:[RTE_train/RTE_gold/loss=6.45e-01, RTE_valid/RTE_gold/accuracy=6.53e-01] model:[train/all/loss=6.45e-01, train/all/lr=5.00e-05]
Saving model at iteration 1.03 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.653
[1.13 epo]: RTE:[RTE_train/RTE_gold/loss=5.87e-01, RTE_valid/RTE_gold/accuracy=6.61e-01] model:[train/all/loss=5.87e-01, train/all/lr=5.00e-05]
Saving model at iteration 1.13 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.661
[1.23 epo]: RTE:[RTE_train/RTE_gold/loss=5.81e-01, RTE_valid/RTE_gold/accuracy=6.35e-01] model:[train/all/loss=5.81e-01, train/all/lr=5.00e-05]
[1.33 epo]: RTE:[RTE_train/RTE_gold/loss=5.76e-01, RTE_valid/RTE_gold/accuracy=6.39e-01] model:[train/all/loss=5.76e-01, train/all/lr=5.00e-05]
[1.44 epo]: RTE:[RTE_train/RTE_gold/loss=5.98e-01, RTE_valid/RTE_gold/accuracy=6.53e-01] model:[train/all/loss=5.98e-01, train/all/lr=5.00e-05]
[1.54 epo]: RTE:[RTE_train/RTE_gold/loss=5.71e-01, RTE_valid/RTE_gold/accuracy=6.32e-01] model:[tr

[2.05 epo]: RTE:[RTE_train/RTE_gold/loss=5.09e-01, RTE_valid/RTE_gold/accuracy=6.61e-01] model:[train/all/loss=5.09e-01, train/all/lr=5.00e-05]
[2.15 epo]: RTE:[RTE_train/RTE_gold/loss=3.25e-01, RTE_valid/RTE_gold/accuracy=6.90e-01] model:[train/all/loss=3.25e-01, train/all/lr=5.00e-05]
Saving model at iteration 2.15 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.690
[2.26 epo]: RTE:[RTE_train/RTE_gold/loss=3.13e-01, RTE_valid/RTE_gold/accuracy=6.61e-01] model:[train/all/loss=3.13e-01, train/all/lr=5.00e-05]
[2.36 epo]: RTE:[RTE_train/RTE_gold/loss=4.76e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=4.76e-01, train/all/lr=5.00e-05]
[2.46 epo]: RTE:[RTE_train/RTE_gold/loss=4.65e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=4.65e-01, train/all/lr=5.00e-05]
[2.56 epo]: RTE:[RTE_train/RTE_gold/loss=4.49e-01, RTE_valid/RTE_gold/accuracy=6.82e-01] model:[train/all/loss=4.49e-01, train/all/lr=5.00e-05]
[2.67 epo]: RTE:[RTE_train/RTE_gold/loss=4.87

[3.08 epo]: RTE:[RTE_train/RTE_gold/loss=2.91e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=2.91e-01, train/all/lr=5.00e-05]
[3.18 epo]: RTE:[RTE_train/RTE_gold/loss=2.79e-01, RTE_valid/RTE_gold/accuracy=7.00e-01] model:[train/all/loss=2.79e-01, train/all/lr=5.00e-05]
Saving model at iteration 3.18 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.700
[3.28 epo]: RTE:[RTE_train/RTE_gold/loss=3.57e-01, RTE_valid/RTE_gold/accuracy=7.00e-01] model:[train/all/loss=3.57e-01, train/all/lr=5.00e-05]
[3.38 epo]: RTE:[RTE_train/RTE_gold/loss=2.62e-01, RTE_valid/RTE_gold/accuracy=6.90e-01] model:[train/all/loss=2.62e-01, train/all/lr=5.00e-05]
[3.49 epo]: RTE:[RTE_train/RTE_gold/loss=3.36e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=3.36e-01, train/all/lr=5.00e-05]
[3.59 epo]: RTE:[RTE_train/RTE_gold/loss=4.97e-01, RTE_valid/RTE_gold/accuracy=6.68e-01] model:[train/all/loss=4.97e-01, train/all/lr=5.00e-05]
[3.69 epo]: RTE:[RTE_train/RTE_gold/loss=3.73

...

[5.03 epo]: RTE:[RTE_train/RTE_gold/loss=2.90e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=2.90e-01, train/all/lr=5.00e-05]
[5.13 epo]: RTE:[RTE_train/RTE_gold/loss=2.06e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=2.06e-01, train/all/lr=5.00e-05]
[5.23 epo]: RTE:[RTE_train/RTE_gold/loss=1.89e-01, RTE_valid/RTE_gold/accuracy=6.61e-01] model:[train/all/loss=1.89e-01, train/all/lr=5.00e-05]
[5.33 epo]: RTE:[RTE_train/RTE_gold/loss=3.83e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=3.83e-01, train/all/lr=5.00e-05]
[5.44 epo]: RTE:[RTE_train/RTE_gold/loss=1.98e-01, RTE_valid/RTE_gold/accuracy=6.43e-01] model:[train/all/loss=1.98e-01, train/all/lr=5.00e-05]
[5.54 epo]: RTE:[RTE_train/RTE_gold/loss=4.63e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=4.63e-01, train/all/lr=5.00e-05]
[5.64 epo]: RTE:[RTE_train/RTE_gold/loss=2.26e-01, RTE_valid/RTE_gold/accuracy=6.39e-01] model:[train/all/loss=2.26e-01, train/all/lr=5.

[6.05 epo]: RTE:[RTE_train/RTE_gold/loss=2.80e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=2.80e-01, train/all/lr=5.00e-05]
[6.15 epo]: RTE:[RTE_train/RTE_gold/loss=2.80e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=2.80e-01, train/all/lr=5.00e-05]
[6.26 epo]: RTE:[RTE_train/RTE_gold/loss=2.75e-01, RTE_valid/RTE_gold/accuracy=6.25e-01] model:[train/all/loss=2.75e-01, train/all/lr=5.00e-05]
[6.36 epo]: RTE:[RTE_train/RTE_gold/loss=3.29e-01, RTE_valid/RTE_gold/accuracy=6.46e-01] model:[train/all/loss=3.29e-01, train/all/lr=5.00e-05]
[6.46 epo]: RTE:[RTE_train/RTE_gold/loss=3.12e-01, RTE_valid/RTE_gold/accuracy=6.82e-01] model:[train/all/loss=3.12e-01, train/all/lr=5.00e-05]
[6.56 epo]: RTE:[RTE_train/RTE_gold/loss=3.00e-01, RTE_valid/RTE_gold/accuracy=6.57e-01] model:[train/all/loss=3.00e-01, train/all/lr=5.00e-05]
[6.67 epo]: RTE:[RTE_train/RTE_gold/loss=3.65e-01, RTE_valid/RTE_gold/accuracy=6.43e-01] model:[train/all/loss=3.65e-01, train/all/lr=5.

[7.08 epo]: RTE:[RTE_train/RTE_gold/loss=1.78e-01, RTE_valid/RTE_gold/accuracy=6.86e-01] model:[train/all/loss=1.78e-01, train/all/lr=5.00e-05]
[7.18 epo]: RTE:[RTE_train/RTE_gold/loss=4.03e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=4.03e-01, train/all/lr=5.00e-05]
[7.28 epo]: RTE:[RTE_train/RTE_gold/loss=3.21e-01, RTE_valid/RTE_gold/accuracy=6.43e-01] model:[train/all/loss=3.21e-01, train/all/lr=5.00e-05]
[7.38 epo]: RTE:[RTE_train/RTE_gold/loss=2.73e-01, RTE_valid/RTE_gold/accuracy=6.79e-01] model:[train/all/loss=2.73e-01, train/all/lr=5.00e-05]
[7.49 epo]: RTE:[RTE_train/RTE_gold/loss=4.10e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=4.10e-01, train/all/lr=5.00e-05]
[7.59 epo]: RTE:[RTE_train/RTE_gold/loss=3.42e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=3.42e-01, train/all/lr=5.00e-05]
[7.69 epo]: RTE:[RTE_train/RTE_gold/loss=2.94e-01, RTE_valid/RTE_gold/accuracy=6.50e-01] model:[train/all/loss=2.94e-01, train/all/lr=5.

[8.10 epo]: RTE:[RTE_train/RTE_gold/loss=4.09e-01, RTE_valid/RTE_gold/accuracy=6.61e-01] model:[train/all/loss=4.09e-01, train/all/lr=5.00e-05]
[8.21 epo]: RTE:[RTE_train/RTE_gold/loss=2.57e-01, RTE_valid/RTE_gold/accuracy=6.57e-01] model:[train/all/loss=2.57e-01, train/all/lr=5.00e-05]
[8.31 epo]: RTE:[RTE_train/RTE_gold/loss=1.87e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=1.87e-01, train/all/lr=5.00e-05]
[8.41 epo]: RTE:[RTE_train/RTE_gold/loss=3.28e-01, RTE_valid/RTE_gold/accuracy=6.25e-01] model:[train/all/loss=3.28e-01, train/all/lr=5.00e-05]
[8.51 epo]: RTE:[RTE_train/RTE_gold/loss=4.15e-01, RTE_valid/RTE_gold/accuracy=6.39e-01] model:[train/all/loss=4.15e-01, train/all/lr=5.00e-05]
[8.62 epo]: RTE:[RTE_train/RTE_gold/loss=3.24e-01, RTE_valid/RTE_gold/accuracy=6.32e-01] model:[train/all/loss=3.24e-01, train/all/lr=5.00e-05]
[8.72 epo]: RTE:[RTE_train/RTE_gold/loss=3.12e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=3.12e-01, train/all/lr=5.

[9.03 epo]: RTE:[RTE_train/RTE_gold/loss=4.34e-01, RTE_valid/RTE_gold/accuracy=6.39e-01] model:[train/all/loss=4.34e-01, train/all/lr=5.00e-05]
[9.13 epo]: RTE:[RTE_train/RTE_gold/loss=2.20e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=2.20e-01, train/all/lr=5.00e-05]
[9.23 epo]: RTE:[RTE_train/RTE_gold/loss=3.34e-01, RTE_valid/RTE_gold/accuracy=6.79e-01] model:[train/all/loss=3.34e-01, train/all/lr=5.00e-05]
[9.33 epo]: RTE:[RTE_train/RTE_gold/loss=3.65e-01, RTE_valid/RTE_gold/accuracy=6.57e-01] model:[train/all/loss=3.65e-01, train/all/lr=5.00e-05]
[9.44 epo]: RTE:[RTE_train/RTE_gold/loss=2.76e-01, RTE_valid/RTE_gold/accuracy=6.61e-01] model:[train/all/loss=2.76e-01, train/all/lr=5.00e-05]
[9.54 epo]: RTE:[RTE_train/RTE_gold/loss=3.18e-01, RTE_valid/RTE_gold/accuracy=6.79e-01] model:[train/all/loss=3.18e-01, train/all/lr=5.00e-05]
[9.64 epo]: RTE:[RTE_train/RTE_gold/loss=3.71e-01, RTE_valid/RTE_gold/accuracy=6.43e-01] model:[train/all/loss=3.71e-01, train/all/lr=5.

### Evaluate baseline slices

In [13]:
import copy
eval_payload = copy.deepcopy(payloads_slice[1])

# NOTE: we need to retarget slices to the original RTE head
for label_name in ['RTE_slice:dash_semicolon', 'RTE_slice:more_people', 'RTE_slice:BASE']:
    for slice_head_type in ['ind', 'pred']:
        eval_payload.retarget_labelset(f'{label_name}:{slice_head_type}', 'RTE')

labelset 'RTE_slice:dash_semicolon:ind' -> task 'RTE' (originally, RTE_slice:dash_semicolon:ind).
labelset 'RTE_slice:dash_semicolon:pred' -> task 'RTE' (originally, RTE_slice:dash_semicolon:pred).
labelset 'RTE_slice:more_people:ind' -> task 'RTE' (originally, RTE_slice:more_people:ind).
labelset 'RTE_slice:more_people:pred' -> task 'RTE' (originally, RTE_slice:more_people:pred).
labelset 'RTE_slice:BASE:ind' -> task 'RTE' (originally, RTE_slice:BASE:ind).
labelset 'RTE_slice:BASE:pred' -> task 'RTE' (originally, RTE_slice:BASE:pred).


In [14]:
eval_payload

Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolon:ind': 'RTE', 'RTE_slice:dash_semicolon:pred': 'RTE', 'RTE_slice:more_people:ind': 'RTE', 'RTE_slice:more_people:pred': 'RTE', 'RTE_slice:BASE:ind': 'RTE', 'RTE_slice:BASE:pred': 'RTE'}], split=valid)

In [15]:
model.score(eval_payload)

{'RTE/RTE_valid/RTE_gold/accuracy': 0.7003610108303249,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 0.5126353790613718,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.646551724137931,
 'RTE/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.5631768953068592,
 'RTE/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.75,
 'RTE/RTE_valid/RTE_slice:BASE:ind/accuracy': 0.4223826714801444,
 'RTE/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.7003610108303249}

### Initialize and train slice model

In [16]:
from metal.mmtl.slicing.tasks import convert_to_slicing_tasks
slicing_tasks = convert_to_slicing_tasks(tasks_slice)
slicing_tasks

Modifying RTE out_features from 2 -> 1
Modifying RTE_slice:dash_semicolon:ind out_features from 2 -> 1
Modifying RTE_slice:dash_semicolon:pred out_features from 2 -> 1
Modifying RTE_slice:more_people:ind out_features from 2 -> 1
Modifying RTE_slice:more_people:pred out_features from 2 -> 1
Modifying RTE_slice:BASE:ind out_features from 2 -> 1
Modifying RTE_slice:BASE:pred out_features from 2 -> 1


[BinaryClassificationTask(name=RTE, loss_multiplier=1.00, slice_head_type=None),
 BinaryClassificationTask(name=RTE_slice:dash_semicolon:ind, loss_multiplier=1.00, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:dash_semicolon:pred, loss_multiplier=1.00, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:more_people:ind, loss_multiplier=1.00, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:more_people:pred, loss_multiplier=1.00, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:BASE:ind, loss_multiplier=1.00, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:BASE:pred, loss_multiplier=1.00, slice_head_type=pred)]

In [17]:
slice_model = SliceModel(slicing_tasks, seed=SEED, verbose=False)

In [18]:
## %%time
trainer = MultitaskTrainer(seed=SEED)
trainer.train_model(
    slice_model,
    payloads_slice,
    task_metrics=[
        "RTE/RTE_train/RTE_gold/loss", 
        "RTE/RTE_train/RTE_slice:BASE:ind/loss",
        "RTE/RTE_train/RTE_slice:dash_semicolon:ind/loss", 
        "RTE/RTE_train/RTE_slice:more_people:ind/loss",
        "RTE/RTE_train/RTE_slice:BASE:pred/loss",
        "RTE/RTE_train/RTE_slice:dash_semicolon:pred/loss", 
        "RTE/RTE_train/RTE_slice:more_people:pred/loss",
        
        "RTE/RTE_valid/RTE_gold/accuracy",
        "RTE/RTE_valid/RTE_slice:BASE:ind/accuracy", 
        "RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy", 
        "RTE/RTE_valid/RTE_slice:more_people:ind/accuracy",
        "RTE/RTE_valid/RTE_slice:BASE:pred/accuracy", 
        "RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy", 
        "RTE/RTE_valid/RTE_slice:more_people:pred/accuracy",
    ],
    checkpoint_metric="RTE/RTE_valid/RTE_gold/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=1e-5,
#     l2=1e-3,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=20,
    progress_bar=True,
    checkpoint_best=True,
    checkpoint_cleanup=False
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 312 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/zzweng/metal/logs/2019_04_22/15_45_46/config.json


[0.10 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=5.27e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=6.87e-01, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=4.51e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=7.27e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.17e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=8.21e-01, RTE_valid/RTE_slice:more_people:ind/accuracy=7.65e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=7.68e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=8.33e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=6.11e-01, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=6.91e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=4.69e-01] model:[train/all/loss=7.03e-01, train/all/lr=1.00e-05]
Saving model at iteration 0.10 with best (max) score RTE/RTE_valid/RTE_go

[1.03 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.66e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=6.76e-01, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=5.81e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=6.99e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.17e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=1.82e-01, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=1.03e-01, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=6.96e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=4.69e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=6.88e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] model:[train/all/loss=4.89e-01, train/all/lr=1.00e-05]
[1.13 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/acc

[1.95 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.62e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=6.24e-01, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=6.32e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=6.95e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=6.12e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=1.27e-01, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=3.96e-02, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=6.89e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=6.06e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=6.59e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] model:[train/all/loss=4.53e-01, train/all/lr=1.00e-05]



[2.05 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.55e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=6.20e-01, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=6.14e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=6.82e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.60e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=8.93e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=3.55e-02, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=6.86e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=5.67e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=7.07e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] model:[train/all/loss=4.42e-01, train/all/lr=1.00e-05]
[2.15 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/acc

...

[18.05 epo]: RTE:[RTE_train/RTE_gold/loss=6.91e-01, RTE_valid/RTE_gold/accuracy=6.17e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=2.85e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=1.00e+00] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=9.48e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=6.97e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=4.79e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=6.39e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=2.58e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.69e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=2.39e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] model:[train/all/loss=2.60e-01, train/all/lr=1.00e-05]
[18.15 epo]: RTE:[RTE_train/RTE_gold/loss=6.91e-01, RTE_valid/RTE_gold/a

[18.97 epo]: RTE:[RTE_train/RTE_gold/loss=6.91e-01, RTE_valid/RTE_gold/accuracy=5.78e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=2.14e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.96e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=2.20e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.60e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=1.42e-01, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=9.18e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=4.66e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=6.17e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=3.22e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=6.67e-01] model:[train/all/loss=2.63e-01, train/all/lr=1.00e-05]



[19.08 epo]: RTE:[RTE_train/RTE_gold/loss=6.91e-01, RTE_valid/RTE_gold/accuracy=6.14e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=2.59e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=1.00e+00] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=3.51e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.60e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=6.37e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=4.99e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=8.13e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=4.83e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=6.43e-01] model:[train/all/loss=2.60e-01, train/all/lr=1.00e-05]
[19.18 epo]: RTE:[RTE_train/RTE_gold/loss=6.91e-01, RTE_valid/RTE_gold/a

[20.0 epo]: RTE:[RTE_train/RTE_gold/loss=6.90e-01, RTE_valid/RTE_gold/accuracy=6.03e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=2.41e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=1.00e+00] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=3.68e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.60e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=1.36e-01, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=4.65e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=6.67e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=1.08e-02, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=4.62e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=6.43e-01] model:[train/all/loss=2.73e-01, train/all/lr=1.00e-05]

[20.00 epo]: model:[train/all/lr=1.00e-05] RTE:[RTE_valid/RTE_gold/accur

{'RTE/RTE_train/RTE_gold/accuracy': 0.6261044176706827,
 'RTE_slice:dash_semicolon:ind/RTE_train/RTE_slice:dash_semicolon:ind/accuracy': 1.0,
 'RTE_slice:dash_semicolon:pred/RTE_train/RTE_slice:dash_semicolon:pred/accuracy': 0.86839481555334,
 'RTE_slice:more_people:ind/RTE_train/RTE_slice:more_people:ind/accuracy': 0.97429718875502,
 'RTE_slice:more_people:pred/RTE_train/RTE_slice:more_people:pred/accuracy': 0.796875,
 'RTE_slice:BASE:ind/RTE_train/RTE_slice:BASE:ind/accuracy': 1.0,
 'RTE_slice:BASE:pred/RTE_train/RTE_slice:BASE:pred/accuracy': 0.7734939759036145,
 'RTE/RTE_valid/RTE_gold/accuracy': 0.6389891696750902,
 'RTE_slice:dash_semicolon:ind/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 1.0,
 'RTE_slice:dash_semicolon:pred/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.603448275862069,
 'RTE_slice:more_people:ind/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.9566787003610109,
 'RTE_slice:more_people:pred/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.4166666666

#### Did we improve?

In [19]:
%%time
slice_model.score(payloads_slice[1])

CPU times: user 1.62 s, sys: 224 ms, total: 1.85 s
Wall time: 1.83 s


{'RTE/RTE_valid/RTE_gold/accuracy': 0.6389891696750902,
 'RTE_slice:dash_semicolon:ind/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 1.0,
 'RTE_slice:dash_semicolon:pred/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.603448275862069,
 'RTE_slice:more_people:ind/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.9566787003610109,
 'RTE_slice:more_people:pred/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.4166666666666667,
 'RTE_slice:BASE:ind/RTE_valid/RTE_slice:BASE:ind/accuracy': 1.0,
 'RTE_slice:BASE:pred/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.6642599277978339}

In [20]:
slice_model.score(eval_payload)

{'RTE/RTE_valid/RTE_gold/accuracy': 0.6389891696750902,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 0.6173285198555957,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.5172413793103449,
 'RTE/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.23465703971119134,
 'RTE/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.3333333333333333,
 'RTE/RTE_valid/RTE_slice:BASE:ind/accuracy': 0.8014440433212996,
 'RTE/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.6389891696750902}